In [ ]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [ ]:
# Example Bloom Filter

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

In [ ]:
# Example Bloom Filter 2

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = double_sha256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

In [ ]:
# Example Bloom Filter 3

from helper import double_sha256, hash160

bit_field_size = 10
bit_field = [0] * bit_field_size

phrase1 = b'hello world'
h1 = double_sha256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = double_sha256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

In [ ]:
# Example BIP0037 Bloom Filter

from helper import murmur3
from bloomfilter import BIP37_CONSTANT

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

### Exercise 1

#### 1.1 Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Make [this test](/edit/session8/bloomfilter.py) pass.

In [ ]:
# Exercise 1.1
from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

# bit_field_size is 8 * field_size
# create a bit field with the appropriate size

# for each item you want to add to the filter
    # iterate function_count number of times
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        # set the bit to be h mod the bit_field_size
        # set the bit_field at the index bit to be 1
# print the bit field converted to bytes using bit_field_to_bytes in hex

In [ ]:
# Exercise 1.2

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_add'))

### Exercise 2

#### 2.1. Make [this test](/edit/session8/bloomfilter.py) pass.

#### 2.2. Do the following:

* Connect to a testnet node
* Load a filter for your testnet address
* Send a request for transactions from the block which had your previous testnet transaction
* Receive the merkleblock and tx messages.

In [ ]:
# Exercise 2.1

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_filterload'))

In [ ]:
# Exercise 2.2

from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import decode_base58, double_sha256, little_endian_to_int
from merkleblock import MerkleBlock
from network import (
    FILTERED_BLOCK_DATA_TYPE, 
    GetDataMessage,
    SimpleNode,
)

block_hash = bytes.fromhex('<look this up>')
passphrase = b'<your email address> <optional phrase>'  # FILL THIS IN
secret = little_endian_to_int(double_sha256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = -1  # FILL THIS IN

# get the hash160 of the address, which is what we need to add to our filter
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
# add the h160 to the bloom filter

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
# complete the handshake
# load the bloom filter with the filterload command
# create a getdata message
# add a FILTERED_BLOCK_DATA_TYPE for the block hash above
# send a getdata message for a FILTERED_BLOCK_DATA_TYPE
# wait for the merkleblock command
# wait for the tx command

### Exercise 3

#### 3.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [ ]:
# Exercise 3.1

reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_is_valid'))

### Exercise 4

#### 4.1. You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [ ]:
# Exercise 4.1

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import (
    decode_base58,
    double_sha256,
    encode_varint,
    little_endian_to_int, 
    read_varint,
    SIGHASH_ALL,
)
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script
from tx import Tx, TxIn, TxOut

last_block_hex = '0000000000000b8e8529996b47b8c3dd13c70adfaf94278883e05bba6ae9159a'
passphrase = b'jimmy@programmingblockchain.com secret passphrase'  # FILL THIS IN
secret = little_endian_to_int(double_sha256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# get the hash160 of the address, which is what we need to add to our filter
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
# add the h160 to the bloom filter

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
# complete the handshake
# load the bloom filter with the filterload command

# set start block to last_block from above
# create GetHeadersMessage with this start block
# send a getheaders message with the start block

# wait for the headers message
# get the stream from the headers
# parse the headers message
# store the last block as None
# initialize the GetDataMessage
# loop through the blocks in the headers
    # check that the proof of work on the block is valid
    # check that this block's prev_block is the last block
    # add FILTERED_BLOCK_DATA_TYPE block.hash() to GetDataMessage
    # set the last block to the current hash
# send the getdata message

# initialize prev_tx and prev_index to None
# initialize prev_amount to None
# while prev_tx is None 
    # wait for the merkleblock or tx commands
    # initialize the stream from the envelope
    # if we have the merkleblock command
        # parse the MerkleBlock
        # check that the MerkleBlock is valid
    # else we have the tx command
        # parse the tx
        # loop through the enumerated tx outs (enumerate(t.tx_outs))
            # if our output has the same address as our address (addr) we found it
                # we found our utxo. set prev_tx, prev_index, prev_amount
                # set the cache so we don't look up the tx in a block explorer
                # Use: TxIn.set_cache(prev_tx, t.serialize())
# create tx_in
# calculate the output amount (prev_amount - fee)
# create a new TxOut to target_script.serialize() with the output amount
# create a new transaction (testnet=True)
# sign the transaction at input 0 with your private key and SIGHASH_ALL
# serialize and hex to see what it looks like
# send this signed transaction on the network
# wait a sec so this message goes through to the other node time.sleep(1) 
# now ask for this transaction from the other node
# create a GetDataMessage
# ask for our transaction by adding its hash and TX_DATATYPE to the message
# send the message
# now wait for a response
envelope = node.wait_for_commands({b'tx', b'reject'})
if envelope.command == b'tx':
    print(envelope.payload.hex())
    print(t.hash().hex())
else:
    print(envelope.payload)